In [1]:
from torchvision.models import resnet50, ResNet50_Weights

In [4]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\Leonard/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100.0%
